<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/ANN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import zipfile  # For extracting zip files
import os  # For verifying extraction
from google.colab import files  # For file upload

# Step 1: Upload the .zip file from your local system
uploaded = files.upload()  # This will prompt the user to select the .zip file

# Extract the file name from the uploaded file
zip_file_path = next(iter(uploaded))  # Get the name of the uploaded file
print(f"Uploaded file: {zip_file_path}")

# Step 2: Extract the .zip file
extract_path = '/content/dataset'  # Path to extract the dataset

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)  # Extract to '/content/dataset'

# Verify extraction by listing the files in the extraction directory
print("Extracted files:", os.listdir(extract_path))  # Should show 'train' and 'validation' folders

# Step 3: Define the CNN model architecture
image_size = (128, 128)  # Resize images to 128x128
batch_size = 32

# Build the CNN model
model = Sequential([
    # 1. Rescaling layer to normalize pixel values between 0 and 1
    Rescaling(1.0 / 255, input_shape=(128, 128, 3)),

    # 2. First convolutional layer
    Conv2D(16, (3, 3), activation='relu'),

    # 3. First max-pooling layer
    MaxPooling2D(pool_size=(2, 2)),

    # 4. Second convolutional layer
    Conv2D(32, (3, 3), activation='relu'),

    # 5. Second max-pooling layer
    MaxPooling2D(pool_size=(2, 2)),

    # 6. Dropout layer to prevent overfitting
    Dropout(0.20),

    # 7. Third convolutional layer
    Conv2D(64, (3, 3), activation='relu'),

    # 8. Third max-pooling layer
    MaxPooling2D(pool_size=(2, 2)),

    # 9. Dropout layer to prevent overfitting
    Dropout(0.25),

    # 10. Flatten layer to convert 2D feature maps to 1D
    Flatten(),

    # 11. First dense (fully connected) layer with 128 units
    Dense(128, activation='relu'),

    # 12. Second dense (fully connected) layer with 64 units
    Dense(64, activation='relu'),

    # 13. Output layer with 2 units (softmax for binary classification)
    Dense(2, activation='softmax')
])

# Step 4: Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary labels
    metrics=['accuracy']
)

# Step 5: Load training and validation data using ImageDataGenerator
train_dir = os.path.join(extract_path, 'train')  # Path to 'train' folder
validation_dir = os.path.join(extract_path, 'validation')  # Path to 'validation' folder

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize images
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to 'train' folder
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' if binary classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,  # Path to 'validation' folder
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' if binary classification
)

# Step 6: Train the model
history = model.fit(
    train_generator,
    epochs=50,  # Adjust epochs based on performance
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Step 7: Save the trained model
model.save('/content/drive/MyDrive/alzheimers_cnn_model_with_aug.h5')


Saving archive.zip to archive (1).zip
Uploaded file: archive (1).zip
Extracted files: ['Alzheimer_MRI_4_classes_dataset']


/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/train'